In [1]:
import matplotlib.pyplot as plt
import pathlib
import numpy as np
import os
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow import keras
from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from tensorflow.keras.layers import GlobalAveragePooling2D,Convolution2D,BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from keras.applications.densenet import preprocess_input

2023-06-14 14:56:51.110884: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-14 14:56:51.883923: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-14 14:56:51.883974: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-06-14 14:56:51.971480: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-14 14:56:53.838222: W tensorflow/stream_executor/platform/de

In [4]:
#from google.colab import drive
#drive.mount("content")

Mounted at content


In [2]:
batch_size = 32
img_height = 150
img_width = 150

In [6]:
pwd

'/home/antoine_d/code/SuperMacintosh/dermasaaj'

In [7]:
test_ds = tf.keras.utils.image_dataset_from_directory('/home/antoine_d/code/SuperMacintosh/dermasaaj/raw_data/data/test'
                                        ,batch_size=batch_size
                                        ,label_mode='categorical'
                                        ,image_size=(img_height,img_width))

Found 2606 files belonging to 8 classes.


2023-06-14 14:59:49.145365: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-06-14 14:59:49.146148: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-06-14 14:59:49.146180: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-L4G0OHE): /proc/driver/nvidia/version does not exist
2023-06-14 14:59:49.150416: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
class_names = test_ds.class_names
class_names

['AK', 'BCC', 'BKL', 'DF', 'MEL', 'NV', 'SCC', 'VASC']

In [9]:
from tensorflow import keras
from keras.applications.densenet import preprocess_input

def new_preproc(X,y):
  return preprocess_input(X),y

# train_preproc_ds = train_ds.map(new_preproc)
# val_preproc_ds = val_ds.map(new_preproc)
test_preproc_ds = test_ds.map(new_preproc)

In [10]:
!gsutil ls gs://derma-data/models/

gs://derma-data/models/
gs://derma-data/models/DERMA_20230608-153259.h5
gs://derma-data/models/DenseNet121_best_model.keras
gs://derma-data/models/DenseNet121_binary_model.keras
gs://derma-data/models/my_first_model.keras


In [ ]:
#!gsutil cp gs://derma-data/models/DenseNet121_best_model.keras

CommandException: Wrong number of arguments for "cp" command.


In [14]:
new_model = tf.keras.models.load_model('my_best_model.keras')

TypeError: weight_decay is not a valid argument, kwargs should be empty  for `optimizer_experimental.Optimizer`.

In [8]:
new_model.evaluate(test_preproc_ds)

82/82 [==============================] - 401s 4s/step - loss: 0.8102 - accuracy: 0.7571 - recall: 0.7375


[0.8102347254753113, 0.7570989727973938, 0.7375288009643555]

In [ ]:
# normalization_layer = layers.Rescaling(1./255)
# normalized_first_test_ds = first_test_ds.map(lambda x, y: (normalization_layer(x), y))

In [16]:
#Predict with new data - new_model
mole_URL = 'https://storage.googleapis.com/derma-data/additional_data/MEL/ISIC_0016123.JPG'
mole_path = tf.keras.utils.get_file(origin=mole_URL)

img = tf.keras.utils.load_img(mole_path, target_size=(img_height, img_width))

img_array = tf.keras.utils.img_to_array(img)
img_array = preprocess_input(img_array)    #preprocess input DenseNet

img_array = tf.expand_dims(img_array, 0) # Create a batch


predictions = new_model.predict(img_array)


print(predictions[0])
print(class_names)
print(f"This image most likely belongs to {class_names[np.argmax(predictions)]} with a probability of {round(100 * np.max(predictions),2)} %")



1/1 [==============================] - 0s 106ms/step
[4.1299220e-10 6.6396999e-09 4.2311507e-04 1.3008018e-07 9.9922705e-01
 3.4966768e-04 4.9275073e-10 7.6423063e-09]
['AK', 'BCC', 'BKL', 'DF', 'MEL', 'NV', 'SCC', 'VASC']
This image most likely belongs to MEL with a probability of 99.92 %


In [ ]:
#Predict with new data - first_model
# mole_URL = 'https://storage.googleapis.com/derma-data/additional_data/AK/ISIC_6434212.JPG'
# mole_path = tf.keras.utils.get_file(origin=mole_URL)

# img = tf.keras.utils.load_img(
#     mole_path, target_size=(128, 128)
# )

# img = normalization_layer(img) #First model

# img_array = tf.keras.utils.img_to_array(img)
# img_array = tf.expand_dims(img_array, 0) # Create a batch

# predictions = first_model.predict(img_array)
# score = tf.nn.softmax(predictions[0])

# print(
#     "This image most likely belongs to {} with a {:.2f} percent confidence."
#     .format(class_names[np.argmax(score)], 100 * np.max(score))
# )

1/1 [==============================] - 0s 37ms/step
This image most likely belongs to BCC with a 35.37 percent confidence.
